# OpenAI Responses API with function call

This example demonstrates how to use function calling with OpenAI's Responses API.

**Note: Current support for the OpenAI Responses API is limited to `initiate_chat` with a two-agent chat. Future releases will included expanded support for group chat and the `run` interfaces.**

## Install AG2 and dependencies

To be able to run this notebook, you will need to install AG2 with the `openai` extra.
````{=mdx}
:::info Requirements
Install `ag2` with 'openai' extra:
```bash
pip install ag2[openai]
```
For more information, please refer to the [installation guide](https://docs.ag2.ai/latest/docs/user-guide/basic-concepts/installing-ag2).
:::
````

In [ ]:
from autogen import ConversableAgent, LLMConfig

# Configure the LLM
llm_config = LLMConfig(
    api_type="responses",
    model="gpt-4o-mini",
    temperature=0.2,
)

# -------------------------------------------------------------
# A simple function we want the assistant to call as a *tool*
# -------------------------------------------------------------


def add_numbers(a: float, b: float) -> str:
    """Return the sum of *a* and *b* as a string (for easy printing)."""
    return str(a + b)


with llm_config:
    # Create a *User* agent that will EXECUTE tools.
    user = ConversableAgent(
        name="User",
        llm_config=False,  # No LLM needed for the executor agent
        human_input_mode="NEVER",  # Run fully autonomously for the demo
    )

    # Create an *Assistant* agent that will REASON about when to call tools.
    assistant = ConversableAgent(
        name="Assistant",
        # ⚠️  Ensure you have a valid OPENAI_API_KEY exported in your env.
        human_input_mode="NEVER",
    )

    # ------------------------------------------------------------------
    # Register the tool for both agents:
    #   • assistant → can *propose* it via a tool call (register_for_llm)
    #   • user      → will actually *execute* it (register_for_execution)
    # ------------------------------------------------------------------
    assistant.register_for_llm(description="Add two numbers together.")(add_numbers)
    user.register_for_execution()(add_numbers)

    # ------------------------------------------------------------------
    # Kick-off a short chat. The assistant should recognise that calling the
    # tool is the best way to fulfil the request and trigger it automatically.
    # ------------------------------------------------------------------
    user.initiate_chat(
        assistant,
        message="Hi, please add 42 and 58.",
        max_turns=2,
        clear_history=True,
    )